# The output characteristics of esm-if1 are obtained. Because the environment does not match, the running environment of colab is given here

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/MyDrive/esm_if/

In [ ]:
# Colab environment setup

# Install the correct version of Pytorch Geometric.
import os
import numpy as np
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric
!pip install -q jsonlines
# Install esm
!pip install -q git+https://github.com/facebookresearch/esm.git

# Install biotite
!pip install -q biotite

In [ ]:
## Verify that pytorch-geometric is correctly installed
import torch_geometric
import torch_sparse
import jsonlines
from torch_geometric.nn import MessagePassing

In [ ]:
import esm
model, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
model = model.eval()

In [ ]:
all_num = 6918
now_num = 0
with jsonlines.open('esmif_output.jsonl', mode='a') as writer:    
    with open("esmif_data_my.jsonl", "r+", encoding="utf8") as f:
        for old_data in jsonlines.Reader(f):
            now_num=now_num+1
            print("\r", end="")
            print("Download progress: {}% ".format(now_num/all_num*100), end="")
            coords = np.array(old_data['coords'],dtype=np.float32)
            feat = esm.inverse_folding.util.get_encoder_output(model, alphabet, coords)
            feat = feat.cpu()
            feat = feat.detach().numpy()
            feat = feat.tolist()
            item = {}
            item['name'] = old_data['name']
            item['features'] = feat
            writer.write(item)

# After the output is obtained, the final input is obtained using the output file and ProtGVP_input

In [4]:
import jsonlines
import numpy as np
import pandas as pd
import os

In [5]:
feat_list = {}
now_num = 0
with open("esmif6A_part1.jsonl", "r+", encoding="utf8") as ff:
    for old_data in jsonlines.Reader(ff):
        feat_list[old_data['name']] = old_data['features']
with open("esmif6A_part2.jsonl", "r+", encoding="utf8") as ff:
    for old_data in jsonlines.Reader(ff):
        feat_list[old_data['name']] = old_data['features']
with open("esmif6A_part3.jsonl", "r+", encoding="utf8") as ff:
    for old_data in jsonlines.Reader(ff):
        feat_list[old_data['name']] = old_data['features']
        


In [6]:
len_feat_list = len(feat_list)       
with jsonlines.open('esmif6A_mynet.jsonl', mode='w') as writer:   
    with open("../gvp_data6A.jsonl", "r+", encoding="utf8") as f:
        for gvp_data in jsonlines.Reader(f):
            now_num=now_num+1
            print("\r", end="")
            print("Download progress: {}% ".format(now_num/len_feat_list*100), end="")
            name = gvp_data['name']
            item = {}
            try:
                item['pro_feature'] = feat_list[name]
            except:
                continue
            item['name'] = gvp_data['name'] 
            item['seq'] =  gvp_data['seq']
            item['pro_mask'] = gvp_data['pro_mask']
            item['pep_feature'] = gvp_data['pep_feature']
            item['pep_mask'] = gvp_data['pep_mask']
            item['label'] = gvp_data['label']
            writer.write(item)

Download progress: 96.21050384286934%    